# Finetuning을 통해 좋아요가 많이 눌린 프로필 데이터를 활용해 더 좋은 프로필 만들기

### 전략
- self-learning
  - 유저의 feedback을 반영하여 스스로 개선하는 시스템
- liked를 많이 받은 자기소개를 gold 데이터로하여 프로필 정보가 들어오면 liked 가 많이 눌린 자기소개를 만들 수 있도록 finetuning 한다.

## Pseudo Data 만들기

In [1]:
import pandas as pd
import random
import json

In [2]:
df = pd.read_json("./profile_db.jsonl", orient='records', lines=True)

In [3]:
df.head()

,name,age,gender,job,bio,keywords
0,김태현,30,남자,소프트웨어 엔지니어,코드와 커피를 사랑하는 개발자입니다. 일상에서 영감을 받아 새로운 프로젝트를 만드는...,"[코딩, 커피, 여행, 사진]"
1,이수민,28,여자,그래픽 디자이너,"색감과 디자인으로 세상을 아름답게 만드는 것을 꿈꾸는 디자이너입니다. 예술과 음악,...","[디자인, 예술, 음악, 독서]"
2,정민호,32,남자,의사,사람들의 건강을 책임지는 의사입니다. 일이 아닐 때는 다양한 스포츠를 즐기며 건강을...,"[건강, 스포츠, 독서, 여행]"
3,최유리,26,여자,영어 강사,언어를 통해 세상과 소통하는 것을 좋아합니다. 여행을 다니며 새로운 문화를 경험하는...,"[언어, 소통, 여행, 문화]"
4,박준수,34,남자,마케팅 매니저,브랜드의 가치를 높이는 일에 흥미를 느낍니다. 책과 영화를 통해 마케팅에 대한 영감...,"[마케팅, 책, 영화, 브랜딩]"


In [4]:
df['liked'] = df.apply(lambda row: random.randint(0, 100), axis=1)

In [5]:
df.head()

,name,age,gender,job,bio,keywords,liked
0,김태현,30,남자,소프트웨어 엔지니어,코드와 커피를 사랑하는 개발자입니다. 일상에서 영감을 받아 새로운 프로젝트를 만드는...,"[코딩, 커피, 여행, 사진]",88
1,이수민,28,여자,그래픽 디자이너,"색감과 디자인으로 세상을 아름답게 만드는 것을 꿈꾸는 디자이너입니다. 예술과 음악,...","[디자인, 예술, 음악, 독서]",40
2,정민호,32,남자,의사,사람들의 건강을 책임지는 의사입니다. 일이 아닐 때는 다양한 스포츠를 즐기며 건강을...,"[건강, 스포츠, 독서, 여행]",68
3,최유리,26,여자,영어 강사,언어를 통해 세상과 소통하는 것을 좋아합니다. 여행을 다니며 새로운 문화를 경험하는...,"[언어, 소통, 여행, 문화]",89
4,박준수,34,남자,마케팅 매니저,브랜드의 가치를 높이는 일에 흥미를 느낍니다. 책과 영화를 통해 마케팅에 대한 영감...,"[마케팅, 책, 영화, 브랜딩]",48


In [6]:
df_sorted = df.sort_values(by=["liked"], ascending=False)
df_sorted.head()

,name,age,gender,job,bio,keywords,liked
27,유진아,30,여자,의사,"사람의 몸과 마음을 치유하는 것에 가치를 두는 의사입니다. 건강한 생활, 요가와 명...","[건강, 치유, 요가, 명상]",98
13,한지은,26,여자,간호사,바쁜 병원 생활 속에서도 웃음을 잃지 않는 간호사입니다. 여행과 사진 찍기를 통해 ...,"[간호, 여행, 사진]",96
16,서준호,35,남자,사업가,도전을 두려워하지 않는 사업가입니다. 와인과 좋은 음악은 저의 취미이자 휴식입니다.,"[사업, 도전, 와인, 음악]",94
29,오민아,33,여자,연구원,세상을 바꾸는 새로운 발견에 기여하고 싶은 연구원입니다. 과학적 호기심과 탐구 정신...,"[연구, 호기심, 등산, 독서]",93
12,정민수,31,남자,교사,아이들에게 꿈을 심어주는 초등학교 선생님입니다. 책 읽기와 글쓰기로 하루를 마무리하...,"[교육, 독서, 글쓰기]",91


In [7]:
# select top 20
df_top = df_sorted[:20]

In [8]:
def sample_to_messages(sample):
    s = sample
    profile = f"이름: {s['name']}\n나이: {s['age']}\n성별: {s['gender']}\n직업: {s['job']}\n키워드: {', '.join(s['keywords'])}"
    bio = sample['bio']
    
    msgs = {"messages": [{"role": "system", "content": "유저의 프로필을 참고해서 소개팅에 어울리는 멋진 자기소개 만들어줘"},
                          {"role": "user", "content": profile},
                          {"role": "assistant", "content": bio}
                         ]}
    return msgs

In [9]:
msgs_list = []

for _, sample in df_top.iterrows():
    msgs = sample_to_messages(sample)
    msgs_list.append(msgs)

In [10]:
msgs_list[0]

{'messages': [{'role': 'system',
   'content': '유저의 프로필을 참고해서 소개팅에 어울리는 멋진 자기소개 만들어줘'},
  {'role': 'user',
   'content': '이름: 유진아\n나이: 30\n성별: 여자\n직업: 의사\n키워드: 건강, 치유, 요가, 명상'},
  {'role': 'assistant',
   'content': '사람의 몸과 마음을 치유하는 것에 가치를 두는 의사입니다. 건강한 생활, 요가와 명상을 일상에 녹여내며, 사람들과의 깊은 대화를 즐깁니다.'}]}

In [11]:
# JSONL 파일 생성 함수
def create_jsonl(msgs_list, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for msgs in msgs_list:
            json_line = json.dumps(msgs, ensure_ascii=False)
            f.write(json_line + '\n')

In [12]:
n_total = len(msgs_list)
n_train = int(n_total * 0.6)
n_valid = int(n_total*0.2)

In [13]:
train_msgs_list = msgs_list[:n_train]
valid_msgs_list = msgs_list[n_train:n_train + n_valid]
test_msgs_list = msgs_list[n_train + n_valid:]

In [14]:
len(train_msgs_list), len(valid_msgs_list), len(test_msgs_list)

(12, 4, 4)

In [15]:
# 훈련 및 검증 데이터셋을 JSONL 파일로 변환
create_jsonl(train_msgs_list, 'profile_gen_train.jsonl')
create_jsonl(valid_msgs_list, 'profile_gen_valid.jsonl')
create_jsonl(test_msgs_list, 'profile_gen_test.jsonl')

## 모델 Finetuning 하기

In [16]:
from openai import OpenAI

In [17]:
client = OpenAI()

## Upload File

In [18]:
# 최소 10개 샘플 이상 필요
train_file = client.files.create(
  file=open("profile_gen_train.jsonl", "rb"),
  purpose="fine-tune"
)

In [19]:
train_file.id

'file-Ts9akwRVBQKq5aaXDcqmGk'

In [20]:
valid_file = client.files.create(
  file=open("profile_gen_valid.jsonl", "rb"),
  purpose="fine-tune"
)

In [21]:
valid_file.id

'file-H5c2qx7vhHZUgBg2Nfs3n7'

## Finetuning

### Finetuning job 제출하기

In [22]:
job = client.fine_tuning.jobs.create(
  training_file=train_file.id,
  validation_file=valid_file.id, 
  model="gpt-3.5-turbo-1106",
  hyperparameters={
    "n_epochs": 3 # default: 3
  }
)

In [23]:
job.id

'ftjob-TfThlnKS3kg0ftUeYsIpdxr4'

In [24]:
print("Job ID:", job.id)
print("Status:", job.status)

Job ID: ftjob-TfThlnKS3kg0ftUeYsIpdxr4
Status: validating_files


### 현재 Finetuning 상태 가져오기


In [25]:
job = client.fine_tuning.jobs.retrieve(job.id)

In [26]:
job.dict()

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_43414/127413622.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  job.dict()


{'id': 'ftjob-TfThlnKS3kg0ftUeYsIpdxr4',
 'created_at': 1744705680,
 'error': {'code': None, 'message': None, 'param': None},
 'fine_tuned_model': None,
 'finished_at': None,
 'hyperparameters': {'batch_size': 'auto',
  'learning_rate_multiplier': 'auto',
  'n_epochs': 3},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'fine_tuning.job',
 'organization_id': 'org-qzBJNqx2R9Pz1HmKd4Zh0Dmj',
 'result_files': [],
 'seed': 110332418,
 'status': 'validating_files',
 'trained_tokens': None,
 'training_file': 'file-Ts9akwRVBQKq5aaXDcqmGk',
 'validation_file': 'file-H5c2qx7vhHZUgBg2Nfs3n7',
 'estimated_finish': None,
 'integrations': [],
 'metadata': None,
 'method': {'dpo': None,
  'supervised': {'hyperparameters': {'batch_size': 'auto',
    'learning_rate_multiplier': 'auto',
    'n_epochs': 3}},
  'type': 'supervised'},
 'user_provided_suffix': None}

In [27]:
print("Job ID:", job.id)
print("Status:", job.status)

Job ID: ftjob-TfThlnKS3kg0ftUeYsIpdxr4
Status: validating_files


### 학습 과정 확인

In [28]:
# List up to 10 events from a fine-tuning job
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job.id, limit=10)
events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-TfThlnKS3kg0ftUeYsIpdxr4
Validating training file: file-Ts9akwRVBQKq5aaXDcqmGk and validation file: file-H5c2qx7vhHZUgBg2Nfs3n7


## Finetuning된 모델 Inference하기

In [29]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [30]:
infer_model = "ft:gpt-3.5-turbo-1106:personal::SOMETHING"

In [31]:
llm = ChatOpenAI(model=infer_model)

In [32]:
example_profile = """\
이름: 김태현
나이: 27
성별: 남자
직업: 프로그래머
키워드: 코딩, 등산, 커피"""
    

In [33]:
keyword_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "유저의 프로필을 참고해서 소개팅에 어울리는 멋진 자기소개 만들어줘"),
        ("human", "{input}" )
    ]
)

In [34]:
extract_keyword_chain = keyword_prompt_template | llm | StrOutputParser()

In [35]:
extract_keyword_chain.invoke({"input": example_profile})

'안녕하세요, 저는 김태현이라고 합니다. 프로그래머로 일하고 있는 27살 남성입니다. 코딩을 좋아하며 주말에는 등산을 즐깁니다. 커피를 사랑하며 새로운 카페를 찾아다니는 것을 취미로 삼고 있어요. 지적인 호흡을 공유하며 함께 새로운 경험을 만들어 나가고 싶습니다. 같이 코딩하고 커피마시며 여행가면서 등산하는 친구를 찾고 있어요. 함께 새로운 경험을 만들어보고 싶은 분이 계시다면 저를 찾아주세요!'